In [1]:
import pymongo
import pandas as pd

In [2]:
user = 'Estudante'
pwd = 'Estudante20202'
connection_string = 'mongodb+srv://{}:{}@bdrnr.mw4lf.mongodb.net/bdrnr_db?retryWrites=true&w=majority'.format(user, pwd)
client = pymongo.MongoClient(connection_string)
db = client.bdrnr_db

In [3]:
resultado = db.G01.find()
df = pd.DataFrame(list(resultado))
df.head()

,_id,MES_ANO_ACIDENTE,DATA_ACIDENTE,TIPO_DO_ACIDENTE,UF_MUNIC_ACIDENTE,AGENTE_CAUSADOR_ACIDENTE,ORIGEM_DE_CADASTRAMENTO_CAT,EMITENTE_CAT,DATA_EMISSAO_CAT,ESPECIE_DO_BENEFICIO,...,DESC_OCUPACAO,CID_10,SUBCATEGORIA,CAT_CID_10,CATEGORIA,CNAE_EMP,CNAE_DENOMINACAO,COD_MUNIC,MUNICIPIO,POPULACAO
0,5fad600a382b8ee116082397,2019-11-01,2019-11-01,Tipico,Tocantins,Metal - Inclui Liga,Internet,Empregador,2019-12-10,Pa,...,Trabalhador de servicos de limpeza e conservac...,C448,Neoplasia maligna da pele com lesao invasiva,C44,Outras neoplasias malignas da pele,8121,Limpeza em predios e em domicilios,430460,Canoas,344957
1,5fad600a382b8ee116082398,2020-01-01,2020-01-09,Tipico,NaN,"Politriz, Lixadora,",Internet,Empregador,2020-01-10,Pa,...,Soldador,C765,Neoplasia maligna do membro inferior,C76,Neoplasia maligna de outras localizacoes e de ...,5250,Atividades relacionadas a organizacao do trans...,431560,Rio Grande,210005
2,5fad600a382b8ee116082399,2019-12-01,2019-12-11,Tipico,NaN,Ferramenta Portatil,Internet,Empregador,2019-12-12,Pa,...,Tecnico florestal,D092,Carcinoma in situ do olho,D09,Carcinoma in situ de outras localizacoes e das...,210,Producao florestal - florestas plantadas,430190,Barra do Ribeiro,13423
3,5fad600a382b8ee11608239a,2019-09-01,2019-09-25,Tipico,NaN,Chao - Superficie Ut,Internet,Empregador,2019-10-01,Pa,...,Inspetor de servicos de transportes rodoviario...,E639,Deficiencia nutricional nao especificada,E63,Outras deficiencias nutricionais,7112,Servicos de engenharia,431490,Porto Alegre,1479101
4,5fad600a382b8ee11608239b,2020-02-01,2020-02-28,Tipico,NaN,Gas Carbonico (Dioxi,Internet,Empregador,2020-03-02,Pa,...,Auxiliar de enfermagem,G430,Enxaqueca sem aura [enxaqueca comum],G43,Enxaqueca,8610,Atividades de atendimento hospitalar,431490,Porto Alegre,1479101


1- Para nascidos neste século (DATA_NASCIMENTO > "2001"), listar AGENTE_CAUSADOR_ACIDENTE, DATA_NASCIMENTO e SEXO, ordenados por DATA_NASCIMENTO descendente, SEXO e AGENTE_CAUSADOR_ACIDENTE.

In [41]:
resultado = db.G01.find( { 'DATA_NASCIMENTO': { '$gt': '2001' } },
                         { 'AGENTE_CAUSADOR_ACIDENTE':1,
                           'DATA_NASCIMENTO':1,
                           'SEXO':1,
                           '_id':0 }
                       ).sort([('DATA_NASCIMENTO', -1),
                               ('SEXO', 1),
                               ('AGENTE_CAUSADOR_ACIDENTE', 1)])

df = pd.DataFrame(list(resultado))
df.head(10)

,AGENTE_CAUSADOR_ACIDENTE,SEXO,DATA_NASCIMENTO
0,"Veiculo, Nic",Masculino,2004-07-14
1,Queda de Pes. em Mes,Feminino,2003-01-27
2,Empilhadeira,Masculino,2002-04-13
3,"Caixa, Engradado, Ca",Masculino,2002-01-17
4,"Tesoura, Tesourao- F",Masculino,2002-01-03
5,Superficie e Estrutu,Masculino,2001-09-11
6,Metal - Inclui Liga,Masculino,2001-09-09
7,Escada Permanente Cu,Feminino,2001-07-28
8,Escada Permanente Cu,Feminino,2001-06-29
9,"Faca, Facao- Ferrame",Masculino,2001-05-27


2- Ordenados por número de acidentes em ordem decrescente, listar MUNICIPIO, POPULACAO e número de acidentes, dos 20 municípios com mais acidentes.

In [42]:
resultado = db.G01.aggregate([
                        {"$group":{"_id": {"MUNICIPIO":"$MUNICIPIO", 
                                           "POPULACAO":"$POPULACAO"},
                                   "NUMACIDS": {"$sum":1}}},
                        {"$sort":{"NUMACIDS" : -1}},
                        {"$limit":20} ])

print("{:20}\t{:10}\t{:20}".format("Município", "População", "Número de Acidentes" ))
print("{:_<20}\t{:_<10}\t{:_<20}".format("_", "_", "_"))
for documento in resultado:    
    print("{:20}\t{:10}\t{:20}".format(
            documento["_id"]["MUNICIPIO"],
            documento["_id"]["POPULACAO"],
            documento["NUMACIDS"]))

Município           	População 	Número de Acidentes 
____________________	__________	____________________
Porto Alegre        	   1479101	                 297
Caxias do Sul       	    504069	                 107
Passo Fundo         	    201767	                  63
Canoas              	    344957	                  43
Santa Maria         	    280505	                  40
Erechim             	    105059	                  35
Lajeado             	     82951	                  34
Sao Leopoldo        	    234947	                  31
Gravatai            	    279398	                  29
Santa Cruz do Sul   	    129427	                  26
Marau               	     43435	                  22
Novo Hamburgo       	    246452	                  20
Rio Grande          	    210005	                  19
Cachoeirinha        	    129307	                  19
Bento Gonçalves     	    119049	                  17
Carazinho           	     61949	                  16
Santa Rosa          	     72919	              

3- Ordenados em ordem decrescente por número de acidentes, listar as 15 primeiras denominações das atividades econômicas (CNAE_DENOMINACAO), municípios e número de acidentes. (dica, agrupar por CNAE_DENOMINACAO e MUNICIPIO)

In [44]:
resultado = db.G01.aggregate([
                        {"$group":{"_id": {"CNAE_DENOMINACAO":"$CNAE_DENOMINACAO", 
                                           "MUNICIPIO":"$MUNICIPIO"},
                                   "NUMACIDS": {"$sum":1}}},
                        {"$sort":{"NUMACIDS" : -1}},
                        {"$limit":15} ])

print("{:90}\t{:20}\t{:20}".format("Atividades Econômicas", "Município", "Número de Acidentes" ))
print("{:_<90}\t{:_<20}\t{:_<20}".format("_", "_", "_"))
for documento in resultado:    
    print("{:90}\t{:20}\t{:20}".format(
            documento["_id"]["CNAE_DENOMINACAO"],
            documento["_id"]["MUNICIPIO"],
            documento["NUMACIDS"]))

Atividades Econômicas                                                                     	Município           	Número de Acidentes 
__________________________________________________________________________________________	____________________	____________________
Atividades de atendimento hospitalar                                                      	Porto Alegre        	                 132
Atividades de atendimento hospitalar                                                      	Passo Fundo         	                  22
Abate de suinos, aves e outros pequenos animais                                           	Lajeado             	                  18
Atividades de atendimento hospitalar                                                      	Caxias do Sul       	                  16
Atividades de Correio                                                                     	Porto Alegre        	                   9
Fabricacao de cabines, carrocerias e reboques para veiculos automotor

4- Ordenados por mês/ano, denominações das atividades econômicas e número de acidentes, filtrando por agente causador de acidente, sexo e atividade econômica. (escolham uma trinca (agente causador de acidente, sexo e atividade econômica) que permita a consulta retornar entre 5 e 30 linhas).

In [59]:
resultado = db.G01.aggregate([
                        {"$match":{"AGENTE_CAUSADOR_ACIDENTE":"Agente Infeccioso ou",
                                   "SEXO":"Feminino", 
                                   "CNAE_DENOMINACAO":"Atividades de atendimento hospitalar",
                                  }},
                        {"$group":{"_id": {"AGENTE_CAUSADOR_ACIDENTE":"$AGENTE_CAUSADOR_ACIDENTE",
                                           "SEXO":"$SEXO",
                                           "CNAE_DENOMINACAO":"$CNAE_DENOMINACAO", 
                                           "MES_ANO_ACIDENTE":"$MES_ANO_ACIDENTE"},
                                   "NUMACIDS": {"$sum":1}}},
                        {"$sort":{"MES_ANO_ACIDENTE":1,
                                  "CNAE_DENOMINACAO":1,
                                  "NUMACIDS":1}},
                        {"$limit":15} ])

print("{:40}\t{:20}\t{:20}".format("Atividades Econômicas", "Mês/Ano do Acidente", "Número de Acidentes" ))
print("{:_<40}\t{:_<20}\t{:_<20}".format("_", "_", "_"))
for documento in resultado:    
    print("{:40}\t{:20}\t{:20}".format(
            documento["_id"]["CNAE_DENOMINACAO"],
            documento["_id"]["MES_ANO_ACIDENTE"],
            documento["NUMACIDS"]))

Atividades Econômicas                   	Mês/Ano do Acidente 	Número de Acidentes 
________________________________________	____________________	____________________
Atividades de atendimento hospitalar    	2020-02-01          	                   2
Atividades de atendimento hospitalar    	2019-12-01          	                   5
Atividades de atendimento hospitalar    	2020-03-01          	                   5
Atividades de atendimento hospitalar    	2020-01-01          	                   6
Atividades de atendimento hospitalar    	2019-10-01          	                   7
Atividades de atendimento hospitalar    	2019-11-01          	                  10


5- Listar os 10 municípios e respectivas populações com as maiores taxas de acidentes por 100.000 habitantes.

In [60]:
resultado = db.G01.aggregate([
                        {"$group":{"_id": {"MUNICIPIO":"$MUNICIPIO", 
                                           "POPULACAO":"$POPULACAO"},
                                  "TAXA_ACIDENTES": {"$divide": [ {"$sum":1}, "$POPULACAO" ] }}},
                        {"$sort":{"NUMACIDS" : -1}},
                        {"$limit":10} ])

print("{:30}\t{:20}\t{:23}".format("Município", "População", "Número de Acidentes" ))
print("{:_<30}\t{:_<20}\t{:_<23}".format("_", "_", "_"))
for documento in resultado:    
    print("{:30}\t{:20}\t{:23}".format(documento["_id"]["MUNICIPIO"],
            documento["_id"]["POPULACAO"],
            documento["NUMACIDS"]))

OperationFailure: The $divide accumulator is a unary operator, full error: {'operationTime': Timestamp(1605844786, 1), 'ok': 0.0, 'errmsg': 'The $divide accumulator is a unary operator', 'code': 40237, 'codeName': 'Location40237', '$clusterTime': {'clusterTime': Timestamp(1605844786, 1), 'signature': {'hash': b'\xb5\xdb\xa4\x0f+\xbb\xf3\xa3~~Nk[\x08\x900\xfcv\x0b\x9d', 'keyId': 6891613512804597763}}}